In [ ]:
# Layer1

import ccxt
import time
import requests

import pandas as pd
import schedule
from datetime import datetime
import subprocess

import sys
import os


def send_telegram_message_v1(message):
    bot_token = ''
    chat_id = ''
    url = f'https://api.telegram.org/bot{bot_token}/sendMessage'
    # params = {'chat_id': chat_id, 'text': message}
    # response = requests.post(url, json=params)

        # Use Markdown formatting to make text bold
    formatted_message = f'{message}'
    params = {'chat_id': chat_id, 'text': formatted_message, 'parse_mode': 'Markdown'}
    response = requests.post(url, json=params)


def send_telegram_message_v2(message):
    bot_token = '6837141712:AAEC2i3M3nLBGLXyKUqvGs3aHKWToq_m-w0'
    chat_id = '735364573'
    url = f'https://api.telegram.org/bot{bot_token}/sendMessage'
    params = {'chat_id': chat_id, 'text': message}
    response = requests.post(url, json=params)

def number_to_alphabet(number):
    last_8_digits = str(number)[-7:]
    digit_to_letter = {'1': 'A', '2': 'B', '3': 'C', '4': 'D', '5': 'E', '6': 'F', '7': 'G', '8': 'H', '9': 'I', '0': 'J'}
    mapped_letters = ''.join(digit_to_letter[digit] for digit in last_8_digits if digit in digit_to_letter)
    
    return mapped_letters


execution_df = pd.read_csv("df_1.csv", index_col = 0).sort_values(by='Capital_Allocation', ascending=False)

# exchange setup
api_key = ''
secret = ''

exchange = ccxt.binance({
    'apiKey': api_key, 
    'secret': secret,
    'enableRateLimit': True,
    'options': {
        'defaultType': 'future'
    }
})

def run_job():
    
    global MDD_pnl
    global MDU_pnl

    message = ""

    # Fetch current positions
    current_positions = exchange.fetch_balance()['info']['positions']
    active_positions = {position['symbol'].replace('USDT', '/USDT'): position for position in current_positions if float(position['positionAmt']) != 0}

    # Initialize totals
    total_pnl = 0
    total_initial_cost = 0
    long_short_net = 0
    fr_net = 0

    pnl_results = {}

    # Aggregating PnL information
    for symbol, row in execution_df.iterrows():
        position_type = row['Position']
        first_letter = position_type[0]
        formatted_symbol = symbol.replace('USDT', '/USDT')

        
        if position_type in ['Long', 'Short'] and formatted_symbol in active_positions:
            position = active_positions[formatted_symbol]
            position_amt = float(position['positionAmt'])
            entry_price = float(position['entryPrice'])
            unrealized_pnl = float(position['unrealizedProfit'])

            total_cost = abs(entry_price * position_amt) 
            percentage_change = (unrealized_pnl / total_cost) * 100 if total_cost != 0 else 0

            # Update totals and store results
            total_pnl += unrealized_pnl
            total_initial_cost += total_cost

            funding_info = exchange.fetch_funding_rate(symbol)
            funding_rate = funding_info['fundingRate'] * 100

            symbol_with_position = f"{symbol}({first_letter}: {round(funding_rate, 4)}%)"
            pnl_results[symbol_with_position] = {
                'pnl': round(unrealized_pnl, 2),
                'percentage_change': round(percentage_change, 2),
                'cost': round(total_cost, 2),
                'amount': position_amt,
                'fr' : round(funding_rate,4),
                'entry_price' : entry_price
            }

    MDD_pnl = min(MDD_pnl, total_pnl)
    MDU_pnl = max(MDU_pnl, total_pnl)

    total_percentage_change = round((total_pnl / total_initial_cost) * 100, 2) if total_initial_cost != 0 else 0
    MDD_total_percentage_change = round((MDD_pnl / total_initial_cost) * 100, 2) if total_initial_cost != 0 else 0
    MDU_total_percentage_change = round((MDU_pnl / total_initial_cost) * 100, 2) if total_initial_cost != 0 else 0

    current_time_unix = int(time.time())

    message += f"🟢 LAYER1 🟢 $A{number_to_alphabet(current_time_unix)} \n \n"

    # Print aggregated PnL results
    for symbol_with_position, info in pnl_results.items():
        message += f"${symbol_with_position.replace('USDT', '')}:   *{info['pnl']}* / {info['cost']}   ( _{info['percentage_change']}%_ )\n"
        long_short_net += info['amount'] * info['entry_price']
        fr_net += info['fr']/100 * info['amount'] * info['entry_price'] * -1 # 양수면 벌고 음수면 잃고
        

    message += f"\n💰 Total = {round(total_pnl,2)} / {round(total_initial_cost,2)} ({total_percentage_change}%)"
    message += f"\n💰 MDD = {round(MDD_pnl,2)} / {round(total_initial_cost,2)} ({MDD_total_percentage_change}%)"
    message += f"\n💰 MDU = {round(MDU_pnl,2)} / {round(total_initial_cost,2)} ({MDU_total_percentage_change}%)"
    message += f"\n💰 Net L/S = {round(long_short_net,2)}"
    message += f"\n💰 Net F/R = {round(fr_net,4)}"


    send_telegram_message_v1(message)

        # 1/2프로 떨어지면 포지션 정리 (마켓으로)
    if total_percentage_change >= 0.35 or total_percentage_change < -1.0:
        message = ""
        USDT_old = float(exchange.fetch_balance()['info']['assets'][next((index for index, asset_info in enumerate(exchange.fetch_balance()['info']['assets']) if asset_info['asset'] == 'USDT'), None)]['walletBalance'])
        for symbol, position in active_positions.items():
            try:
                formatted_symbol = symbol.replace('/USDT', 'USDT')
                side = 'sell' if float(position['positionAmt']) > 0 else 'buy'
                position_amt = abs(float(position['positionAmt']))
                #resp = exchange.set_leverage(leverage=1, symbol=formatted_symbol)
                order = exchange.create_order(formatted_symbol, 'market', side, position_amt)
                
                message += f"🔴 Position cleared for {formatted_symbol}\n"

            except Exception as e:
                message += f"Error clearing position for {formatted_symbol}: {e}\n"                
                # print(f"Error clearing position for {formatted_symbol}: {e}")
                continue  # Proceed to the next iteration
        
        time.sleep(3)
        USDT_new = float(exchange.fetch_balance()['info']['assets'][next((index for index, asset_info in enumerate(exchange.fetch_balance()['info']['assets']) if asset_info['asset'] == 'USDT'), None)]['walletBalance'])
        USDT_percent_change = (USDT_new/USDT_old - 1) * 100
        USDT_pnl = USDT_new - USDT_old

        send_telegram_message_v1(message)
        send_telegram_message_v1(f"PNL: {round(USDT_pnl, 2)} ({round(USDT_percent_change, 2)}%)")
        send_telegram_message_v2(f"💰 Total:     {round(total_pnl,2)} / {round(total_initial_cost,2)} ({total_percentage_change}%) \
                                    \n💰 MDD:     {round(MDD_pnl,2)} / {round(total_initial_cost,2)} ({MDD_total_percentage_change}%) \
                                    \n💰 MDU:     {round(MDU_pnl,2)} / {round(total_initial_cost,2)} ({MDU_total_percentage_change}%) \
                                    \n💰 PNL:       {round(USDT_pnl, 2)} ({round(USDT_percent_change, 2)}%)          \
                                    \n💰 USDT:    {round(USDT_new,2)}  \
                                    \n💰 NO:        $A{number_to_alphabet(current_time_unix)}")
            
        time.sleep(3)
        subprocess.Popen(["python", "pca_execution_L1_v1.py"])
        sys.exit()
    
        
def main():
    while True:
        # Get the current time
        current_time = time.time()

        # Calculate how many seconds to wait until the start of the next minute
        wait_time = 15 - (current_time % 15)

        # Sleep until the start of the next minute
        time.sleep(wait_time)

        # Run the job
        run_job()

MDD_pnl = 0
MDU_pnl = 0


if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        # Log the error
        print(f"Fatal error: {e}. Restarting script.")
        # Consider logging to a file or sending a notification for the error here
        
        # Restart the script
        os.execl(sys.executable, sys.executable, *sys.argv)